# 2007-2008 Minor antigenic variants

The goal of this work is to determine which HA variants are nonsymonous and whether or any of the nonsymonous variants are in putative antigenic regions. I am only looking at minor variants in this analyis and so nonsymonous variants will be identified as variants that change the amino acid a given position compared to the sample's consensus seqeunce (not the plasmid control).

In [15]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.Alphabet import IUPAC



Based on what I have seen in the Fluvac data the we have 1700bp of the HA sequences. All seqeunces start with "ATT" When we blast these sequences in NCBI. This first "codon" algins with the start of the coding sequence "ATG". If I make this change in the plasmid control .fa and then translate the altered string, the resulting protein sequence is an exact match to what is observed in NCBI. It looks like Kari aligned to just the coding region during the primary analysis.

The first step will be to read in the parsed HA consensus sequences. They are in two fasta files named after the run from which they come (1293 & 1304). Here I am reading each sequence, adjusting the start codon, translating, and then saving the sequence names, nucleotide seq object, and protein seq object in separate lists

In [16]:
Ids=[]
nucleotide=[]
protein=[]

for seq_record in SeqIO.parse("/Users/jt/Documents/Analysis/FluVac_paper/data/processed/Run_1293/parsed_fa/HA.fa", "fasta"):
    Ids.append(seq_record.id.split(".")[0])
    seq_record.seq.alphabet=IUPAC.unambiguous_dna
    fixed = list(str(seq_record.seq)) # the sequences all start as ATT they should be ATG - this is likely due to issues of coverage near the end. Fixing this error opens up the proper reading frame and when done to the plasmid control it yeilds a 100% match across all 566aa found in NCBI
    fixed[2]="G"
    fixed=''.join(fixed)
    fixed_record=Seq(fixed,IUPAC.unambiguous_dna)
    nucleotide.append(fixed_record)
    prot=fixed_record.translate()
    protein.append(prot)
for seq_record in SeqIO.parse("/Users/jt/Documents/Analysis/FluVac_paper/data/processed/Run_1304/parsed_fa/HA.fa", "fasta"):
    Ids.append(seq_record.id.split(".")[0])
    seq_record.seq.alphabet=IUPAC.unambiguous_dna
    fixed = list(str(seq_record.seq)) # the sequences all start as ATT they should be ATG - this is likely due to issues of coverage near the end. Fixing this error opens up the proper reading frame and when done to the plasmid control it yeilds a 100% match across all 566aa found in NCBI
    fixed[2]="G"
    fixed=''.join(fixed)
    fixed_record=Seq(fixed,IUPAC.unambiguous_dna)
    nucleotide.append(fixed_record)
    prot=fixed_record.translate()
    protein.append(prot)

/usr/local/lib/python3.5/site-packages/Bio/Seq.py:2071: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


Now I will read in all polymorphisms on HA from the 2007-2008 season.

In [17]:
import pandas as pd
data=pd.read_csv("../data/2007_2008.ha.csv")
df=data.sort_values("Id") # Sorting the data frame by sample name. NOTE : The indeces don't change 
df["pos"]=df["pos"]-1 # Note the pandas data frame is in base 1 while everthing else is in base 0 this fixes that


Now to find the codon of each of these positions we will just need to use a little modular algebra. bp 0-2 are in represent the o aa 3-5 the 1 and so forth. So bp divided by 3 no remander should do the trick. This will give the amino acid position or "aa_pos"

In [18]:
df["aa_pos"]=df["pos"]//3


Here I am looping through the sample names (Id) in the variant csv. For each Id, I am finding the index of that Id in the Id list; This is also the index of the sample in the protein list. There are mulitple variants in some samples - by looping through the Ids in the data frame I will grab each of these instances separately. I am using a counter to keep track of what row of the data frame. I am then taking the appropriate amino acid position and grabing the consensus amino acid for that position in the sample. These amino acids are stored in a list and then added to the data frame in the last line

In [19]:
counter=0
consens=[]
for id in df.Id:
    if str(id) in Ids:
        i=Ids.index(str(id)) # get the index of the sample
        aa_pos=list(df["aa_pos"])[counter]
        aa=str(protein[i])[aa_pos]
        #print(str(i)+ " : " + str(id) + " : "+str(aa_pos) + " : " + aa)
        consens.append(aa)
    else:
        consens.append("NA")
    counter=counter+1
df["aa_consensus"]=consens

Now I am looping through the variants by sample name (as above). This time I am introducing the mutation into the nucleaic acid consensus. I am then retranslating the seqeunce, and grabing the appropriate amino acid. As before each step is saved as a list and in the end the variant amino acid is added to the data frame. If the variant amino acid is different than the consensus, the sample Id, nucleaic acid, and amino acid sequences are also saved in separate lists to be used later (denoted as dn\_*)

In [20]:
counter=0
var_nucleotide=[]
var_Ids=[]
var_protein=[]
var_aa=[]

dn_nucleotide=[]
dn_Ids=[]
dn_protein=[]


for id in df.Id:
    if str(id) in Ids:
        i=Ids.index(str(id)) # get the index of the sample
        pos=list(df["pos"])[counter]
        var=list(df["var"])[counter]
        seq=list(str(nucleotide[i])) # get the consensus and make it a list so we can make the changes
        seq[pos]=var
        seq=''.join(seq)
        seq=Seq(seq,IUPAC.unambiguous_dna)
        var_nucleotide.append(seq)
        prot=seq.translate()
        aa_pos=list(df["aa_pos"])
        aa_pos=aa_pos[counter]
        aa=str(prot)[aa_pos]
        con_aa=list(df["aa_consensus"])[counter]
        if aa !=con_aa:
            dn_protein.append(prot)
            dn_Ids.append(str(id))
        
        #print(str(id) + " : "+str(aa_pos) + " : " + aa + " : " + str(counter))
        var_aa.append(aa)
        var_protein.append(prot)
        var_Ids.append(str(id))
    else:
        var_nucleotide.append("NA")
        var_Ids.append("NA")
        var_protein.append("NA")
        var_aa.append("NA")
    counter=counter+1
df["aa_var"]=var_aa

/usr/local/lib/python3.5/site-packages/Bio/Seq.py:2071: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


## The HA variants 

Id is the sample name, ref is the reference base in the plasmid control, var is the variant in the sample, freq.var the frequency, pos the nucleaic acid postion, aa_pos the amino acid position, aa_consensus the amino acid at the consensus level, and aa_var the variant amino acid.

In [21]:
df[["Id","ref",'var',"freq.var","pos","aa_pos","aa_consensus","aa_var"]]


,Id,ref,var,freq.var,pos,aa_pos,aa_consensus,aa_var
34,3,A,T,0.020529,1114,371,Q,L
6,3,G,A,0.985102,670,223,K,K
7,3,G,G,0.014898,670,223,K,R
33,3,A,A,0.979471,1114,371,Q,Q
2,6,C,A,0.039535,284,94,F,L
3,6,C,C,0.960465,284,94,F,F
30,10,G,G,0.980604,1014,338,V,V
17,10,C,T,0.033524,731,243,S,S
16,10,C,C,0.966476,731,243,S,S
13,10,A,G,0.034423,687,229,I,V


In [22]:
nonsense=df.loc[(df.aa_consensus!=df.aa_var)]
nonsense[["Id","ref",'var',"freq.var","pos","aa_pos","aa_consensus","aa_var"]]

,Id,ref,var,freq.var,pos,aa_pos,aa_consensus,aa_var
34,3,A,T,0.020529,1114,371,Q,L
7,3,G,G,0.014898,670,223,K,R
2,6,C,A,0.039535,284,94,F,L
13,10,A,G,0.034423,687,229,I,V
29,10,G,A,0.019396,1014,338,V,I
32,12,G,T,0.017493,1047,349,A,S
21,14,G,A,0.078873,853,284,R,K
68,17,A,A,0.422878,1613,537,M,I
0,27,G,A,0.030421,192,64,G,S
28,46,A,G,0.030150,879,293,N,D


## H3 numbering system! 

Here I am adpapting the HA number script made by Jesse Bloom of the Bloom lab. Currently his script runs as an exicutable that takes in a file containing variables - sequence, position of interest, and path to aligner. I will adapt this into a function to be used here. I will also have to alter the output. [Currently I am just grabbing some of the text output as formating in his script and saving it in lists to be added to the data frame.


In [23]:
import os
import tempfile
import sys
import subprocess

### Adapted HA_numbering.py - function 

In [24]:
def Align(headers_seqs, progpath, program='PROBCONS', musclegapopen=None):
    """Performs a multiple sequence alignment of two or more sequences.

    By default, the protein sequences are aligned using PROBCONS.  This is
        probably the most accurate alignment program.  However, it is
        slow and consumes large amounts of memory if you are aligning
        a very large number of sequences (typically if you are aligning
        more than several hundred).  In that case, you may prefer to use
        MUSCLE instead.  You can choose between the two with the 'program'
        option.  If you decide to use MUSCLE, you can also align nucleotide
        sequences with this program.

    'headers_seqs' is a list specifying the names of the sequences that we
        want to align.  Each entry is a 2-tuple '(head, seq)' where 'head' is
        a header giving the sequence name and other information (might be empty)
        and 'seq' is a string giving the protein sequence.  The list must have
        at least 2 entries.

    'progpath' should specify a directory containing the alignment program executable,
        either PROBCONS or MUSCLE.  The PROBCONS executable is assumed to have
        the name "probcons" in this directory.  The MUSCLE executable is assumed to
        have the name "muscle" in this directory.

    'program' specifies what program to use for the alignment.  By default, it is
        "PROBCONS".  If you wish to use MUSCLE instead, set it to "MUSCLE".  

    'musclegapopen' sets the MUSCLE gap openining penalty to the specified
        value. By default it is None, meaning we use the MUSCLE default penalty.
        You can also set it to a number; for example -100 will lead to fewer gaps.

    This executable is used to perform a multiple sequence alignment of the proteins
        with the default settings of either PROBCONS or MUSCLE.  The returned variable is a
        new list 'aligned_headers_seqs'.  Each entry is a 2-tuple '(head, aligned_seq)'.  
        'head' has the same meaning as on input (the sequence header) and 
        'aligned_seq' is the aligned sequence, with gaps inserted as '-'
        as appropriate.  Therefore, all of the 'aligned_seq' entries in
        'aligned_headers_seqs' are of the same length.  The entries in 'aligned_headers_seq'
        are in the same order as in the input list 'headers_seqs'.
    """
    if not (isinstance(headers_seqs, list) and len(headers_seqs) >= 2):
        raise ValueError('header_seqs does not specify a list with at least two entries.')
    if not os.path.isdir(progpath):
        raise ValueError("Cannot find directory %s." % progpath)
    if program == 'PROBCONS':
        exe = os.path.abspath("%s/probcons" % progpath) # the executable
    elif program == 'MUSCLE':
        exe = os.path.abspath("%s/muscle" % progpath) # the executable
    else:
        raise ValueError("Invalid value of %s for 'program'." % (str(program)))
    if not os.path.isfile(exe):
        raise IOError("Cannot find executable at %s." % exe)
    currdir = os.getcwd()
    tempdir = tempfile.mkdtemp()
    try:
        # do stuff in a temporary directory
        infile = "%s/in.fasta" % tempdir # input file 
        outfile = "%s/out.fasta" % tempdir # output file 
        WriteFASTA(headers_seqs, infile) # write sequences to the input file
        if program == 'PROBCONS':
            p = subprocess.Popen("%s %s" % (exe, infile), shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE) # run ProbCons
            (output, errors) = p.communicate()
            
            open(outfile, 'wb').write(output)
        elif program == 'MUSCLE':
            if musclegapopen != None:
                p = subprocess.Popen("%s -gapopen %d -in %s -out %s" % (exe, musclegapopen, infile, outfile), shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE) # run MUSCLE
            else:
                p = subprocess.Popen("%s -in %s -out %s" % (exe, infile, outfile), shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE) # run MUSCLE
            (output, errors) = p.communicate()
        try:
            aligned_headers_seqs = ReadFASTA(outfile)
        except:
            sys.stderr.write("Error getting alignment output, error of %s" % errors)
            raise
    finally:
        os.chdir(currdir) # return to the original directory
        for file in os.listdir(tempdir):
            os.remove("%s/%s" % (tempdir, file)) # remove files from temporary directory
        os.rmdir(tempdir) # remove temporary directory
    if len(aligned_headers_seqs) != len(headers_seqs):
        raise ValueError("Did not return the correct number of aligned sequences.")
    # put the aligned sequences in the same order as the input sequences
    n = len(aligned_headers_seqs[0][1]) # length of aligned sequences
    d = dict(aligned_headers_seqs)
    aligned_headers_seqs = []
    for (head, seq) in headers_seqs:
        try:
            alignedseq = d[head]
        except KeyError:
            raise ValueError("After alignment, the following header is missing: %s" % head)
        if len(alignedseq) != n:
            open('errors.temp', 'w').write(errors)
            raise ValueError("Aligned sequence %s is not of length %d: if you are using MUSCLE, you may be running out of memory.  Errors have been written to errors.temp." % (alignedseq, n))
        if len(seq) > n:
            open('errors.temp', 'w').write(errors)
            raise ValueError("Unaligned seq %s is longer than aligned length of %d: if you are using MUSCLE, you many be running out of memory.  Errors have been written to errors.temp." % (seq, n))
        aligned_headers_seqs.append((head, alignedseq))
    return aligned_headers_seqs # return the aligned sequences
    

def StripGapsToFirstSequence(aligned_headers_seqs):
    """Strips gaps from a reference sequence, and all corresponding alignments.

    On input, 'aligned_headers_seqs' should be a set of two or more aligned sequences,
        as would be returned by 'Align'.

    The first sequence in this alignment is taken to correspond to the reference sequence.
        The returned variable is a list similar to aligned_headers_seqs, but with
        all positions corresponding to gaps in this reference sequence stripped away.
        All gaps ('-') characters are removed from this reference sequence.  In addition,
        in all other aligned sequences in 'aligned_headers_seqs', every character at
        the same position as a gap in the reference sequence is removed.  Therefore,
        at the end of this procedure, all of the alignments have the same length
        as the reference sequence with its gaps stripped away.  The headers are 
        unchanged.  The order of sequences in this stripped alignment is also
        unchanged.

    >>> StripGapsToFirstSequence([('s1', '-AT-A-GC'), ('s2', 'AAT-TAGC'), ('s3', '--T-A-GC')])
    [('s1', 'ATAGC'), ('s2', 'ATTGC'), ('s3', '-TAGC')]
    """        
    if not (isinstance(aligned_headers_seqs, list) and len(aligned_headers_seqs) >= 2):
        raise ValueError("aligned_headers_seqs does not specify at least two aligned sequences.")
    (ref_head, ref_seq) = aligned_headers_seqs[0]
    non_strip_positions = [] # positions not to strip away
    stripped_ref_seq = []
    for i in range(len(ref_seq)):
        r = ref_seq[i]
        if r != '-':
            non_strip_positions.append(i)
            stripped_ref_seq.append(r)
    stripped_headers_seqs = [(ref_head, ''.join(stripped_ref_seq))]
    for (ihead, iseq) in aligned_headers_seqs[1 : ]:
        istrippedseq = ''.join([iseq[i] for i in non_strip_positions])
        stripped_headers_seqs.append((ihead, istrippedseq))
    return stripped_headers_seqs


def WriteFASTA(headers_seqs, filename, writable_file=False):
    """Writes sequences to a FASTA file.

    'headers_seqs' is a list of 2-tuples specifying sequences and their
        corresponding headers.  Each entry is the 2-tuple '(header, seq)'
        where 'header' is a string giving the header (without the leading ">"),
        and 'seq' is the corresponding sequence.

    'filename' is a string that specifies the name of the file to which the
        headers and sequences should be written.  If this file already exists,
        it is overwritten. 

    'writable_file' is a Boolean switch specifying that rather than 'filename'
        giving a string specifying the name of a file to which the sequences
        should be written, it instead specifies a writable file object to which
        the sequences should be written.

    The sequences are written to the file in the same order that they are specified
        in 'headers_seqs'.
    """
    assert isinstance(writable_file, bool)
    if writable_file:
        f = filename
    else:
        f = open(filename, 'w')
    for (header, seq) in headers_seqs:
        f.write(">%s\n%s\n" % (header, seq))
    if not writable_file:
        f.close()



def ReadFASTA(fastafile):
    """Reads sequences from a FASTA file.

    'fastafile' should specify the name of a FASTA file.

    This function reads all sequences from the FASTA file.  It returns the
        list 'headers_seqs'.  This list is composed of a 2-tuple '(header, seq)'
        for every sequence entry in FASTA file.  'header' is the header for
        a sequence, with the leading ">" and any trailing spaces removed. 'seq'
        is the corresponding sequence.
    """
    lines = open(fastafile).readlines()
    headers_seqs = []
    header = None
    seq = []
    for line in lines:
        if line[0] == '>':
            if (not header) and (not seq):
                pass # first sequence in file
            elif header and not seq:
                raise ValueError("Empty sequence for %s" % header)
            elif seq and not header:
                raise ValueError("File does not begin with header.")
            else:
                seq = ''.join(seq)
                seq = seq.replace(' ', '')
                headers_seqs.append((header, seq))
            header = line.strip()[1 : ]
            seq = []
        else:
            seq.append(line.strip())
    if (not header) and (not seq):
        pass # first sequence in file
    elif header and not seq:
        raise ValueError("Empty sequence for %s" % header)
    elif seq and not header:
        raise ValueError("File does not begin with header.")
    else:
        seq = ''.join(seq)
        seq = seq.replace(' ', '')
        headers_seqs.append((header, seq))
    return headers_seqs


def GetCorrespondingResidue(seqs, i):
    """Gets the corresponding residue number for two aligned sequences.

    *seqs* is a set of two aligned sequences as *(head, sequence)* 2-tuples.

    *i* is the number of a residue in sequential numbering of *seqs[0]*
    without considering any of the gaps induced by alignment, in 1, 2, ...
    numbering.

    Returns the number of the residue in sequential numbering of *seqs[1]*
    without considering any of the gaps induced by alignment in 1, 2, ...
    numbering. Returns *None* if residue *i* of *seqs[0]* aligns with a
    gap in *seqs[1]*.
    """
    assert len(seqs) == 2
    s1 = seqs[0][1]
    s2 = seqs[1][1]
    assert len(s1) == len(s2)
    assert 1 <= i <= len(s1)
    s1index = s2index = 0
    for j in range(len(s1)):
        if s1[j] != '-':
            s1index += 1
        if s2[j] != '-':
            s2index += 1
        if s1index == i:
            if s2[j] == '-':
                return None
            else:
                return s2index


def find_aligments(aligner,aligner_path,seq,pos): # The positions will be given as base 0 and adjusted to match the convention (base 1) in the funciton
    """Main body of script."""
    print("\nBeginning execution of HA_numbering script.")
    
    # parse arguments
    if aligner == 'probcons':
        alignerpath = aligner_path # redundant
        if not os.path.isdir(alignerpath):
            raise IOError("The directory of %s specified by probconspath does not exist." % (alignerpath))
        prog = 'PROBCONS'
    elif aligner == 'muscle':
        alignerpath = aligner_path # redundant
        if not os.path.isdir(alignerpath):
            raise IOError("The directory of %s specified by musclepath does not exist." % (alignerpath))
        prog = 'MUSCLE'
    else:
        raise IOError("First argument does not specify probcons or muscle")
    ha_sequence = str(seq) # just get the sequence
    sites = pos
    # Define sequences and their numbering conversions.
    # The sequences are in seq_d and keyed by PDB code.
    # The number conversions from sequential numbering of these sequences to
    # th number labels are in label_d and keyed by PDB codes.
    seq_names = ['4HMG', '4JTV']
    seq_d = {}
    label_d = {}
    seq_4hmg_a = \
        'QDLPGNDNSTATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICNNPHRILDGIDCTLIDALLGDPHCDVFQ' +\
        'NETWDLFVERSKAFSNCYPYDVPDYASLRSLVASSGTLEFITEGFTWTGVTQNGGSNACKRGPGSGFFSRLNWLTKSGST' +\
        'YPVLNVTMPNNDNFDKLYIWGIHHPSTNQEQTSLYVQASGRVTVSTRRSQQTIIPNIGSRPWVRGQSSRISIYWTIVKPG' +\
        'DVLVINSNGNLIAPRGYFKMRTGKSSIMRSDAPIDTCISECITPNGSIPNDKPFQNVNKITYGACPKYVKQNTLKLATGM' +\
        'RNVPEKQT'
    seq_4hmg_b = \
        'GLFGAIAGFIENGWEGMIDGWYGFRHQNSEGTGQAADLKSTQAAIDQINGKLNRVIEKTNEKFHQIEKEFSEVEGRIQDL' +\
        'EKYVEDTKIDLWSYNAELLVALENQHTIDLTDSEMNKLFEKTRRQLRENAEEMGNGCFKIYHKCDNACIESIRNGTYDHD' +\
        'VYRDEALNNRFQIKG'
    seq_d['4HMG'] = seq_4hmg_a + seq_4hmg_b
    label_d['4HMG'] = dict([(i + 1, '%d in HA1' % (i + 1)) for i in range(len(seq_4hmg_a))] + [(len(seq_4hmg_a) + i + 1, '%d in HA2' % (i + 1)) for i in range(len(seq_4hmg_b))])
    assert len(seq_d['4HMG']) == len(label_d['4HMG'])
    seq_4jtv_a = \
        'DTLCIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDKHNGKLCKLRGVAPLHLGKCNIAGWILGNPECESLSTASSWSYIV' +\
        'ETPSSDNGTCYPGDFIDYEELREQLSSVSSFERFEIFPKTSSWPNHDSNKGVTAACPHAGAKSFYKNLIWLVKKGNSYPK' +\
        'LSKSYINDKGKEVLVLWGIHHPSTSADQQSLYQNADTYVFVGSSRYSKKFKPEIAIRPKVRDQEGRMNYYWTLVEPGDKI' +\
        'TFEATGNLVVPRYAFAMERNAGSGIIISDTPVHDCNTTCQTPKGAINTSLPFQNIHPITIGKCPKYVKSTKLRLATGLRN' +\
        'I'
    seq_4jtv_b = \
        'GLFGAIAGFIEGGWTGMVDGWYGYHHQNEQGSGYAADLKSTQNAIDEITNKVNSVIEKMNTQFTAVGKEFNHLEKRIENL' +\
        'NKKVDDGFLDIWTYNAELLVLLENERTLDYHDSNVKNLYEKVRSQLKNNAKEIGNGCFEFYHKCDNTCMESVKNGTYDYP' +\
        'KY'
    seq_d['4JTV'] = seq_4jtv_a + seq_4jtv_b
    label_d['4JTV'] = dict([(i + 1, '%d of HA1' % (i + 7)) for i in range(len(seq_4jtv_a))] + [(len(seq_4jtv_a) + i + 1, '%d of HA2' % (i + 1)) for i in range(len(seq_4jtv_b))])
    assert len(seq_d['4JTV']) == len(label_d['4JTV'])

    # make alignments
    print("Making %s alignments..." % prog)
    alignments = {}
    for seqname in seq_names:
        alignments[seqname] = Align([('seq', ha_sequence), (seqname, seq_d[seqname])], alignerpath, prog)
    print("Alignments complete.\n\nHere are the corresponding residue numbers:")
    for site in sites:
        site=site+1 # to correct for difference in base numbering
        if not (1 <= site <= len(ha_sequence)):
            raise ValueError("site %d is outside the valid range for sequential numbering of ha_sequence starting at 1." % site)
        sitestring = ['\nResidue %s%d in sequential numbering of your HA sequence corresponds to:' % (ha_sequence[site - 1], site)]
        for seqname in seq_names:
            i = GetCorrespondingResidue(alignments[seqname], site)
            if i == None:
                sitestring.append('  * an alignment gap in %s' % seqname) 
            else:
                if not (1 <= i <= len(seq_d[seqname])):
                    raise ValueError("Invalid corrresponding residue for %s -- something is wrong with this program" % seqname)
                aa = seq_d[seqname][i - 1]
                
                sitestring.append('  * %s%s in %s' % (aa, label_d[seqname][i], seqname))
        print( '\n'.join(sitestring))
        return([sitestring[1][4:-7],sitestring[2][4:-7]])
        




Now I am looping through all nonsynonymous mutations as before and applying the function above to get the H3 numbering system. The script takes in base 0 positions and handels them accordingly.

In [25]:

counter=0
PDB_4HMG=[]
PDB_4JTV=[]
for id in nonsense.Id:
    if str(id) in dn_Ids:
        i=dn_Ids.index(str(id)) # get the index of the sample
        pos=list(nonsense["aa_pos"])[counter]
        nice=find_aligments('probcons','/Users/jt/probcons/',dn_protein[i],[pos])
        PDB_4HMG.append(nice[0])
        PDB_4JTV.append(nice[1])
    counter=counter+1        
nonsense["PDB_4HMG"] = PDB_4HMG
nonsense["PDB_4JTV"]=PDB_4JTV
    


Beginning execution of HA_numbering script.
Making PROBCONS alignments...
Alignments complete.

Here are the corresponding residue numbers:

Residue L372 in sequential numbering of your HA sequence corresponds to:
  * Q27 in HA2 in 4HMG
  * Q27 of HA2 in 4JTV

Beginning execution of HA_numbering script.
Making PROBCONS alignments...
Alignments complete.

Here are the corresponding residue numbers:

Residue K224 in sequential numbering of your HA sequence corresponds to:
  * R208 in HA1 in 4HMG
  * R211 of HA1 in 4JTV

Beginning execution of HA_numbering script.
Making PROBCONS alignments...
Alignments complete.

Here are the corresponding residue numbers:

Residue L95 in sequential numbering of your HA sequence corresponds to:
  * F79 in HA1 in 4HMG
  * L76 of HA1 in 4JTV

Beginning execution of HA_numbering script.
Making PROBCONS alignments...
Alignments complete.

Here are the corresponding residue numbers:

Residue V230 in sequential numbering of your HA sequence corresponds to:
 

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Final output!! 

In [26]:
print(len(list(nonsense["Id"])))
nonsense[["Id","ref",'var',"freq.var","pos","aa_pos","aa_consensus","aa_var","PDB_4HMG","PDB_4JTV"]]

18


,Id,ref,var,freq.var,pos,aa_pos,aa_consensus,aa_var,PDB_4HMG,PDB_4JTV
34,3,A,T,0.020529,1114,371,Q,L,Q27 in HA2,Q27 of HA2
7,3,G,G,0.014898,670,223,K,R,R208 in HA1,R211 of HA1
2,6,C,A,0.039535,284,94,F,L,F79 in HA1,L76 of HA1
13,10,A,G,0.034423,687,229,I,V,I214 in HA1,K217 of HA1
29,10,G,A,0.019396,1014,338,V,I,V323 in HA1,I327 of HA1
32,12,G,T,0.017493,1047,349,A,S,A5 in HA2,A5 of HA2
21,14,G,A,0.078873,853,284,R,K,R269 in HA1,I273 of HA1
68,17,A,A,0.422878,1613,537,M,I,an alignment gap,an alignment gap
0,27,G,A,0.030421,192,64,G,S,G49 in HA1,G45 of HA1
28,46,A,G,0.030150,879,293,N,D,I278 in HA1,N282 of HA1


The final step will be to use this list to reference the putative antigenic sites to see if any match.



And this saves the output as a csv.

In [27]:
nonsense[["Id","ref",'var',"freq.var","pos","aa_pos","aa_consensus","aa_var","PDB_4HMG","PDB_4JTV"]].to_csv("../data/nonsymom.csv")